In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import fastai
fastai.__version__


In [ ]:
from fastai.text.all import *

In [ ]:
df = pd.read_csv('../input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv')

In [ ]:
#0:positive 1:negative
df

In [ ]:
path = Path('../input/imdb-dataset-sentiment-analysis-in-csv-format')

In [ ]:
dls = TextDataLoaders.from_csv(path=path, csv_fname='Train.csv', text_col='text', label_col='label', valid_pct=0.2)
dls.show_batch()

In [ ]:
dls_lm = TextDataLoaders.from_csv(path=path, csv_fname='Train.csv', text_col='text', label_col='label', valid_pct=0.2,is_lm=True)

In [ ]:
learn = language_model_learner(dls_lm, AWD_LSTM,metrics=[accuracy,Perplexity()])

In [ ]:
learn.fit_one_cycle(1,1e-2)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(10,2e-3)

In [ ]:
learn.model_dir='/kaggle/working'
learn.save_encoder('finetuned')

In [ ]:
learn_cls = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5,metrics=accuracy)

In [ ]:
learn_cls.model_dir='/kaggle/working'
learn_cls = learn_cls.load_encoder('finetuned')

In [ ]:
learn_cls.fit_one_cycle(1, 2e-2)

In [ ]:
learn_cls.freeze_to(-2)
learn_cls.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn_cls.freeze_to(-3)
learn_cls.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn_cls.save('stage-1-94')

In [ ]:
learn_cls.unfreeze()
learn_cls.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

In [ ]:
learn_cls.save('stage-2-94')

In [ ]:
learn_cls.predict('''Never buy!
Phone came with sim inserted!
Used one!
Pack opened state''')

In [ ]:
learn_cls.freeze()
learn_cls.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

In [ ]:
learn_cls.export('/kaggle/output/')